In [159]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns 
import datetime

In [160]:
#Все транзакции с марта 2022 и first_date (первая покупка)
df1 = pd.read_csv('tran_premium_1.csv', sep=";")
df2 = pd.read_csv('tran_premium_2.csv', sep=";") 
df3 = pd.read_csv('tran_premium_3.csv', sep=";") 
df4 = pd.read_csv('tran_premium_4.csv', sep=";") 
df5 = pd.read_csv('tran_premium_5.csv', sep=";") 
df6 = pd.read_csv('tran_premium_6.csv', sep=";") 
df7 = pd.read_csv('tran_premium_7.csv', sep=";") 
df8 = pd.read_csv('tran_premium_8.csv', sep=";") 
df9 = pd.read_csv('tran_premium_9.csv', sep=";") 
df10 = pd.read_csv('tran_premium_10.csv', sep=";") 
df11 = pd.read_csv('tran_premium_11.csv', sep=";") 

result = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]) 

In [161]:
result['first_month'] = pd.to_datetime(result['first_tran']).dt.month
result['month_of_using'] = pd.to_datetime(result['business_dt']).dt.month
result['year_of_using'] = pd.to_datetime(result['business_dt']).dt.year
result = result[((result['month_of_using'] >= 8) & (result['year_of_using'] == 2022)) | 
                ((result['month_of_using'] <= 4) & (result['year_of_using'] == 2023))] 
result

,msisdn,bill_amt,discount_amt,bill_price_amt,bill_tax_amt,business_dt,first_tran,first_month,month_of_using,year_of_using
0,79000222592,165.8333,0.0,199.0,33.1667,2022-08-30,2022-03-24,3,8,2022
1,79062944776,165.8333,0.0,199.0,33.1667,2022-08-03,2022-06-04,6,8,2022
3,79004964626,124.1667,0.0,149.0,24.8333,2022-08-09,2022-08-09,8,8,2022
4,79049078070,165.8333,0.0,199.0,33.1667,2022-08-01,2022-06-02,6,8,2022
5,79064464095,165.8333,0.0,199.0,33.1667,2022-08-12,2022-08-12,8,8,2022
...,...,...,...,...,...,...,...,...,...,...
2742120,79894440252,7.5000,0.0,9.0,1.5000,2023-04-17,2022-09-11,9,4,2023
2742121,79894507036,7.5000,0.0,9.0,1.5000,2023-04-21,2022-12-30,12,4,2023
2742122,79894598013,7.5000,0.0,9.0,1.5000,2023-04-19,2022-03-30,3,4,2023
2742123,79894827398,7.5000,0.0,9.0,1.5000,2023-04-20,2022-10-08,10,4,2023


In [162]:
result.reset_index(drop= True , inplace= True)

In [163]:
#пользователи urent в премиуме
urent = pd.read_csv('ua_urent_2207_2209.csv', sep=",")
urent.drop(urent[urent['month of use'] != '2022-08-01'].index, inplace = True)
urent['isUrent'] = 'Urent users from august 2022'
urent.reset_index(drop= True , inplace= True )
urent = urent.drop(['month of use'], axis=1)
urent.msisdn.nunique()

38730

In [164]:
#Начали пользоваться Premium 
premium_from_8_count = result[result['first_month'] == 8]['msisdn'].nunique()
premium_from_8_count

254959

In [165]:
# Делим все транзакции на 2: кто с юрентом, кто без юрента 
premium_from_8 = result[result['first_month'] == 8]
premium_from_8.reset_index(drop= True , inplace= True)
premium_from_8_with_mark_if_Urent = pd.merge(premium_from_8, urent, on="msisdn", how = 'left')
premium_from_8_with_mark_if_Urent['isUrent'] = premium_from_8_with_mark_if_Urent['isUrent'].fillna(
    'Not urent users from august 2022')
premium_from_8_with_mark_if_Urent['isUrent'].value_counts()

Not urent users from august 2022    1279271
Urent users from august 2022          17812
Name: isUrent, dtype: int64

In [166]:
# Находим: сколько из этих 254959 пользуются только премиумом, а сколько еще и юрентом
premium = premium_from_8_with_mark_if_Urent[premium_from_8_with_mark_if_Urent['isUrent'] == 
                                            'Not urent users from august 2022']['msisdn'].nunique()
premiumAndUremt = premium_from_8_with_mark_if_Urent[premium_from_8_with_mark_if_Urent['isUrent'] == 
                                                    'Urent users from august 2022']['msisdn'].nunique()
print(premium, premiumAndUremt)

251547 3412


In [167]:
#Делаем красивую табличку с августовскими, только с нужными данными: msisdn и isUrent
msisdn_isUrent = premium_from_8_with_mark_if_Urent.drop(['bill_amt', 'discount_amt', 'bill_price_amt', 'bill_tax_amt',
                                                         'year_of_using', 'business_dt', 'first_tran',
                                                        'first_month', 'month_of_using'], axis=1)
msisdn_isUrent.reset_index(drop= True, inplace= True)

In [168]:
#Получаем таблицу с транзакциями после 8 августа до конца апреля с поментками, кто с юрентом, кто без. 
# Если нулл - то не с августа
fin_result_isUrent = pd.merge(result, msisdn_isUrent, on=['msisdn'], how = 'left')

In [169]:
fin_result_isUrent

,msisdn,bill_amt,discount_amt,bill_price_amt,bill_tax_amt,business_dt,first_tran,first_month,month_of_using,year_of_using,isUrent
0,79000222592,165.8333,0.0,199.0,33.1667,2022-08-30,2022-03-24,3,8,2022,NaN
1,79062944776,165.8333,0.0,199.0,33.1667,2022-08-03,2022-06-04,6,8,2022,NaN
2,79004964626,124.1667,0.0,149.0,24.8333,2022-08-09,2022-08-09,8,8,2022,Urent users from august 2022
3,79004964626,124.1667,0.0,149.0,24.8333,2022-08-09,2022-08-09,8,8,2022,Urent users from august 2022
4,79049078070,165.8333,0.0,199.0,33.1667,2022-08-01,2022-06-02,6,8,2022,NaN
...,...,...,...,...,...,...,...,...,...,...,...
33325440,79894440252,7.5000,0.0,9.0,1.5000,2023-04-17,2022-09-11,9,4,2023,NaN
33325441,79894507036,7.5000,0.0,9.0,1.5000,2023-04-21,2022-12-30,12,4,2023,NaN
33325442,79894598013,7.5000,0.0,9.0,1.5000,2023-04-19,2022-03-30,3,4,2023,NaN
33325443,79894827398,7.5000,0.0,9.0,1.5000,2023-04-20,2022-10-08,10,4,2023,NaN


In [171]:
fin_result_isUrent['lifetime'] = (pd.to_datetime(fin_result_isUrent['business_dt']) - 
                                  pd.to_datetime(fin_result_isUrent['first_tran'])).dt.days

In [170]:
cohorts = fin_result_isUrent.pivot_table(index = 'isUrent', values = 'msisdn', aggfunc = 'nunique', columns = ['year_of_using', 'month_of_using'])
cohorts

year_of_using                       2022                                2023  \
month_of_using                        8       9      10     11     12     1    
isUrent                                                                        
Not urent users from august 2022  251547  109622  98799  88441  82915  75897   
Urent users from august 2022        3412    1750   1616   1452   1353   1240   

year_of_using                                          
month_of_using                       2      3      4   
isUrent                                                
Not urent users from august 2022  65766  44973  87444  
Urent users from august 2022       1103    685   1318

In [172]:
fin_result_isUrent.groupby(by="isUrent").mean()

C:\Users\avnebuko\AppData\Local\Temp\20\ipykernel_25676\1249353486.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  fin_result_isUrent.groupby(by="isUrent").mean()


,msisdn,bill_amt,discount_amt,bill_price_amt,bill_tax_amt,first_month,month_of_using,year_of_using,lifetime
isUrent,,,,,,,,,
Not urent users from august 2022,7.940990e+10,77.057488,0.0,92.469008,15.411520,8.0,5.509169,2022.671552,168.756864
Urent users from august 2022,7.939740e+10,91.146292,0.0,109.375573,18.229281,8.0,5.694258,2022.620744,156.459886


In [191]:
#Считаем retention rate Not urent users from august 2022
rr_not_urent1 = round(109622/251547 * 100, 0)
rr_not_urent2 = round(98799/109622 * 100, 0)
rr_not_urent3 = round(88441/98799 * 100, 2)
rr_not_urent4 = round(82915/88441 * 100, 2)
rr_not_urent5 = round(75897/82915 * 100, 2)
rr_not_urent6 = round(65766/75897 * 100, 2)
rr_not_urent7 = round(44973/65766 * 100, 2)
rr_not_urent8 = round(87444/44973 * 100, 2)
retention_rate_just_premium = rr_not_urent1, rr_not_urent2, rr_not_urent3, rr_not_urent4, rr_not_urent5, rr_not_urent6, rr_not_urent7, rr_not_urent8

round(sum(retention_rate_just_premium) / (len(retention_rate_just_premium)),2)
# print('rr_not_urent1 =  {}, rr_not_urent2 = {}, rr_not_urent3 = {}, rr_not_urent4 = {}, rr_not_urent5 = {}, rr_not_urent6 = {}, rr_not_urent7 = {}, rr_not_urent8 = {}'
#       .format(rr_not_urent1, rr_not_urent2, rr_not_urent3, rr_not_urent4, rr_not_urent5, rr_not_urent6, rr_not_urent7, 
#              rr_not_urent8))

94.78

In [189]:
retention_rate_just_premium

(44.0, 90.0, 89.52, 93.75, 91.54, 86.65, 68.38)

In [192]:
#Считаем retention rate Urent users from august 2022
rr_urent1 = round(1750/3412 * 100, 0)
rr_urent2 = round(1616/1750 * 100, 0)
rr_urent3 = round(1452/1616 * 100, 2)
rr_urent4 = round(1353/1452 * 100, 2)
rr_urent5 = round(1240/1353 * 100, 2)
rr_urent6 = round(1103/1240 * 100, 2)
rr_urent7 = round(685/1103 * 100, 2)
rr_urent8 = round(1318/685 * 100, 2)
retention_rate_urent_premium = rr_urent1, rr_urent2, rr_urent3, rr_urent4, rr_urent5, rr_urent6, rr_urent7, rr_urent8

round(sum(retention_rate_urent_premium) / (len(retention_rate_urent_premium)), 2)

95.14

In [193]:
retention_rate_urent_premium

(51.0, 92.0, 89.85, 93.18, 91.65, 88.95, 62.1, 192.41)